In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
pos = pd.read_csv(PATH + 'POS_CASH_balance.csv')

'done'

'done'

In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
merged_col = 'SK_ID_CURR'
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()

# minus
name1 = minus_name('SK_DPD', 'SK_DPD_DEF')
pos[name1] = minus(pos, 'SK_DPD', 'SK_DPD_DEF')

# ratio
numerator = ['SK_DPD', 'SK_DPD_DEF', name1]
denominator = ['MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']
for x in numerator:
    for y in denominator:
        print(x, y)
        pos[ratio_name(x, y)] = ratio(pos, x, y)

# one hot encoding sum
df = df.merge(one_hot(pos, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')

# positive count
for x in ['SK_DPD', 'SK_DPD_DEF', name1]:
    df = df.merge(positive_count(pos, merged_col, x))

# count
df = df.merge(count(pos, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]

# categorical ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)

# numerical cols
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS']
for x in set(pos.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(pos, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')
    
columns = []
for x in df.columns:
    tmp = 'poscash_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns

'done'

SK_DPD MONTHS_BALANCE
SK_DPD CNT_INSTALMENT
SK_DPD CNT_INSTALMENT_FUTURE
SK_DPD_DEF MONTHS_BALANCE
SK_DPD_DEF CNT_INSTALMENT
SK_DPD_DEF CNT_INSTALMENT_FUTURE
SK_DPD_minus_SK_DPD_DEF MONTHS_BALANCE
SK_DPD_minus_SK_DPD_DEF CNT_INSTALMENT
SK_DPD_minus_SK_DPD_DEF CNT_INSTALMENT_FUTURE
onehot_NAME_CONTRACT_STATUS_Active
onehot_NAME_CONTRACT_STATUS_Amortized debt
onehot_NAME_CONTRACT_STATUS_Approved
onehot_NAME_CONTRACT_STATUS_Canceled
onehot_NAME_CONTRACT_STATUS_Completed
onehot_NAME_CONTRACT_STATUS_Demand
onehot_NAME_CONTRACT_STATUS_Returned to the store
onehot_NAME_CONTRACT_STATUS_Signed
onehot_NAME_CONTRACT_STATUS_XNA
positivecount_SK_ID_CURR_SK_DPD
positivecount_SK_ID_CURR_SK_DPD_DEF
positivecount_SK_ID_CURR_SK_DPD_minus_SK_DPD_DEF
SK_DPD_minus_SK_DPD_DEF
SK_DPD_divide_MONTHS_BALANCE
SK_DPD
SK_DPD_DEF_divide_MONTHS_BALANCE
SK_DPD_minus_SK_DPD_DEF_divide_MONTHS_BALANCE
CNT_INSTALMENT
SK_DPD_divide_CNT_INSTALMENT_FUTURE
SK_DPD_minus_SK_DPD_DEF_divide_CNT_INSTALMENT_FUTURE
SK_DPD_minus_SK_

'done'

In [4]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'poscash_onehot_NAME_CONTRACT_STATUS_Active',
       'poscash_onehot_NAME_CONTRACT_STATUS_Amortized debt',
       'poscash_onehot_NAME_CONTRACT_STATUS_Approved',
       'poscash_onehot_NAME_CONTRACT_STATUS_Canceled',
       'poscash_onehot_NAME_CONTRACT_STATUS_Completed',
       'poscash_onehot_NAME_CONTRACT_STATUS_Demand',
       'poscash_onehot_NAME_CONTRACT_STATUS_Returned to the store',
       'poscash_onehot_NAME_CONTRACT_STATUS_Signed',
       'poscash_onehot_NAME_CONTRACT_STATUS_XNA',
       ...
       'poscash_SK_DPD_divide_CNT_INSTALMENT_mean',
       'poscash_SK_DPD_divide_CNT_INSTALMENT_median',
       'poscash_SK_DPD_divide_CNT_INSTALMENT_max',
       'poscash_SK_DPD_divide_CNT_INSTALMENT_min',
       'poscash_SK_DPD_divide_CNT_INSTALMENT_sum', 'poscash_SK_DPD_DEF_mean',
       'poscash_SK_DPD_DEF_median', 'poscash_SK_DPD_DEF_max',
       'poscash_SK_DPD_DEF_min', 'poscash_SK_DPD_DEF_sum'],
      dtype='object', length=101)


(337252, 101)

In [5]:
df.to_pickle(PATH + 'inter/poscash2curr.pkl')
'done'

'done'